In [139]:
import numpy as np 
import pandas as pd
import pulp

In [ ]:
"""
Parameters:
safety stock, maximum warehouse capacity
h: unit holding cost
max_p: production max per month
min_p = production min per month
wc = max inventory capacity per month
ss = min inventory capacity per month
I_0: initial_inventory
c_t: unit production cost in month t
d_t: demand of month t

Variables:
P_t: Amount produced in month t
I_t: Inventory at the end of period t

Constraints:
Inventory Constraints: I_{t-1} + X_t - d_t = I_t (Beginning Inventory)
                    I_t + P_t - d_t >= ss
                    I_t + P_t - d_t <= wc
Capacity Constraints: P_t <= max_p
                      P_t >= min_p

Objective: Min Sum(((2*I_t)+P_t - d_t).5*h*c_t + c_t*P_t)

I_t - I_{t-1}+P{t-1}-D{t-1}=0


"""

In [140]:
filepath = '/Volumes/Elements/khartless/Projects/Data/'
parameters_name = 'Upton.xlsx'

input_data = pd.read_excel(filepath+parameters_name, sheet_name= 'Production', index_col=[0])


In [141]:
initial_inventory = 2750
safety_stock = 1500
max_capacity = 6000
carrying_cost = .015

In [169]:
model = pulp.LpProblem(name='prod_planning_upton', sense=pulp.LpMinimize)

production_variables = pulp.LpVariable.dicts(name='P', indexs=input_data.index,
                                             lowBound=0, cat=pulp.LpContinuous)

inventory_variables = pulp.LpVariable.dicts(name='I', indexs=input_data.index,
                                            lowBound=0, cat=pulp.LpContinuous)

#beginning inventory constraint
for period, value in input_data.iloc[1:].iterrows():
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[period-1] + production_variables[period-1] - inventory_variables[period],
        sense=pulp.LpConstraintEQ,
        name='inv_balance' + str(period),
        rhs=input_data.Demand[period-1]))

# set beginning inventory
model += inventory_variables[1] == initial_inventory

for index, value in production_variables.items():
    model.addConstraint(pulp.LpConstraint(
        e=value,
        sense=pulp.LpConstraintLE,
        name='prod_cap_month_' + str(index),
        rhs=input_data.loc[index].Max_Production))

#minimum ending inventory
for period, value in input_data.iterrows():
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[period] + production_variables[period],
        sense=pulp.LpConstraintGE,
        name='inv_min_month' + str(period),
        rhs= safety_stock + value.Demand))

#maximum ending inventory
for period, value in input_data.iterrows():
    model.addConstraint(pulp.LpConstraint(
        e=inventory_variables[period] + production_variables[period],
        sense=pulp.LpConstraintLE,
        name='inv_cap_month' + str(period),
        rhs= max_capacity + value.Demand))

#Objective: Min Sum(((2*I_t)+P_t - d_t).5*h*c_t + c_t*P_t)

total_holding_cost = pulp.lpSum(((2*inventory_variables[index])+production_variables[index] - row['Demand'])*.5 * carrying_cost * row['Unit_Production_Cost']
                                for index, row in input_data.iterrows())
total_production_cost = pulp.lpSum(row['Unit_Production_Cost'] * production_variables[index]
                                   for index, row in input_data.iterrows())

objective = total_holding_cost + total_production_cost

model.setObjective(objective)

In [170]:
model.solve()
print(f'status: {model.status}, {pulp.LpStatus[model.status]}')


status: 1, Optimal


In [171]:
print(f'objective: ${model.objective.value():,.0f}')

for var in model.variables():
    print(f'{var.name}: {var.value():,.0f}')

for name, constraint in model.constraints.items():
    print(f'{name}: {constraint.value():,.2f}')

objective: $6,209,403
I_1: 2,750
I_2: 5,750
I_3: 4,750
I_4: 2,750
I_5: 1,500
I_6: 2,000
P_1: 4,000
P_2: 3,500
P_3: 4,000
P_4: 4,250
P_5: 4,000
P_6: 3,500
inv_balance2: 0.00
inv_balance3: 0.00
inv_balance4: 0.00
inv_balance5: 0.00
inv_balance6: 0.00
_C1: 0.00
prod_cap_month_1: 0.00
prod_cap_month_2: 0.00
prod_cap_month_3: 0.00
prod_cap_month_4: -250.00
prod_cap_month_5: 0.00
prod_cap_month_6: 0.00
inv_min_month1: 4,250.00
inv_min_month2: 3,250.00
inv_min_month3: 1,250.00
inv_min_month4: 0.00
inv_min_month5: 500.00
inv_min_month6: 0.00
inv_cap_month1: -250.00
inv_cap_month2: -1,250.00
inv_cap_month3: -3,250.00
inv_cap_month4: -4,500.00
inv_cap_month5: -4,000.00
inv_cap_month6: -4,500.00
